In [ ]:
!nvidia-smi

Tue Jun 13 05:14:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |  11799MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%pip install evaluate datasets Xformers
%pip install -q -U bitsandbytes
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/peft.git
%pip install -q -U git+https://github.com/huggingface/accelerate.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
%cd "/content/drive/MyDrive/02 ColabDir/Playground_CausalLM"

/content/drive/MyDrive/02 ColabDir/Playground_CausalLM


In [ ]:
CKPT = "EleutherAI/polyglot-ko-1.3b"
DATASET = "beomi/KoAlpaca-v1.1a"

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

bnbConfig = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = AutoModelForCausalLM.from_pretrained(CKPT,
                                             quantization_config=bnbConfig,
                                             device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Tue Jun 13 05:16:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |  11799MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 1333383168 || trainable%: 0.11796039111242178


In [ ]:
TEST = [
    ("대한민국의 수도는 어디인가요?"),
    ("카페라뗴와 카페오레의 차이점은 무엇인가요?"),
    ("영일만친구들은 코딩을 잘 하나요?"),
]

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model, tokenizer=tokenizer,
                device_map='auto')

result = [i for i in pipe(i[0] for i in TEST)]
result

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

[[{'generated_text': '대는 것이다.'}],
 [{'generated_text': '카레를 먹었다.\u200b\u200b\u200b\u200b\u200b\u200b�'}],
 [{'generated_text': '영(李瑞環)은 그의 아들이다. 그는'}]]

In [ ]:
from datasets import load_dataset
dataset = load_dataset(DATASET)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from pprint import pprint

pprint(dataset)

dataset['train'][0]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})


{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [ ]:
dataset = dataset.map(lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" })
dataset = dataset.map(lambda d: tokenizer(d['text']), batched=True)
dataset['train'][0]['text']

'### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=1,
        output_dir="ckpts",
        optim="paged_adamw_8bit"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.615300
2,2.787400
3,2.602700
4,2.913000
5,2.762000
6,2.826200
7,2.801500
8,2.778200
9,2.766300
10,2.503500


TrainOutput(global_step=10, training_loss=2.7356024265289305, metrics={'train_runtime': 35.0519, 'train_samples_per_second': 1.141, 'train_steps_per_second': 0.285, 'total_flos': 72712711299072.0, 'train_loss': 2.7356024265289305, 'epoch': 0.0})

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from pprint import pprint
from datetime import datetime


model.eval()
model.config.use_cache = True

since = datetime.now()
generated = [model.generate(**tokenizer(t, return_tensors='pt', return_token_type_ids=False),
                max_new_tokens=256,
                early_stopping=True,
                do_sample=True,
                eos_token_id=2,
                ) for t in TEST]
elapsed = (datetime.now()-since).seconds
generated = [tokenizer.decode(x[0]) for x in generated]
for i, j in zip(TEST, generated):
    print("IN: ", end="")
    print(i)
    print("OUT: ", end="")
    print(j)
print(f"Elapsed: {elapsed:04} seconds")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


IN: 대한민국의 수도는 어디인가요?
OUT: 대한민국의 수도는 어디인가요?​​​답 : 서울입니다. 서울서울​​​​​6.​대한민국의 대도시는 어디인가요?​​답 : 대전도​​​7.​대한민국의 지역권은 어디인가요?​​답 : 서울특별시, 경기도, 강원도, 충청남도, 충청북도, 전라남도, 전라북도, 경상북도​​​8.​대한민국의 행정구역과 지방자치는 어떻게 나누어져 있나요?​​답 : ​대한민국의 행정구역은1 중앙정부의 지방행정제도를 근간으로 하여 특별시. 광역시, 도, 시, 군, 자치구로 구분되며, 이에따라 관할구역이 넓은 행정수도인 세종특별자치시의 관할구역이 상대적으로 좁은 편입니다.​​​9.​대한민국의 지방자치는 어떻게 이루어지고 있나요?​답 : ​지방자치의 가장 큰 목적은지역의 문제를 스스로 해결하는 것입니다. 지역내 문제가 발생하면 주민 스스로 스스로 해결해 나아가는 생활자치입니다. ​​​​​10.​대한민국에서 지방자치를 잘하고
IN: 카페라뗴와 카페오레의 차이점은 무엇인가요?
OUT: 카페라뗴와 카페오레의 차이점은 무엇인가요?- 차이는 없습니당 ㅎㅎ;;;;​ ​​[카페 라떼 (Caffe Latte)]​- 카페 라떼는 따뜻한 카페라떼와 우유가 들어간 라떼를 일컫는 말입니다.​- 보통 카페라떼는 '카페라떼'라는 이름이 붙어 있는 카페라떼 보다 진하게 나오기 때문에 우유가 많아서 고소한 맛을 느끼실 수 있습니당 ​​ ​​​​​[카페 오레 (Caffe ore)]​- 카페 라떼에 비해 우유 양이 적어서 카페모카처럼 크림이 올라가 있어요​- 커피가 들어간 카페 라떼가 부담스러우시다면, 카페라떼와 카페 오레의 중간 쯤 되는 '카페오레'를 드시는 걸 추천합니다.​​​​​ [라떼아트의 유래]-라떼아트란 말 그대로 '카페'에서 나오는 에스프레소에 우유가 첨가된 커피를 뜻합니다.​-1970년대부터 유럽과 미국의 바리스타들 사이에서는 커피 주문과 관련
IN: 영일만친구들은 코딩을 잘 하나요?
OUT: 영일만친구들은 코딩을 잘 하나요? 코딩이 뭐죠?​초등학교 선생님과 중학교 선생님의

In [ ]:
model.save_pretrained('./ckpts/sample_peft')

In [ ]:
model.push_to_hub('quantizedCausalTest00')

adapter_model.bin:   0%|          | 0.00/6.31M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/gyulukeyi/quantizedCausalTest00/commit/102822baa4dfe8a8551c0a82512b924284348f7b', commit_message='Upload model', commit_description='', oid='102822baa4dfe8a8551c0a82512b924284348f7b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained('gyulukeyi/quantizedCausalTest00')

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model2 = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                              quantization_config=bnb_config, device_map="auto",)
model2 = PeftModel.from_pretrained(model, 'gyulukeyi/quantizedCausalTest00', )
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,)

model2.eval()

since = datetime.now()
generated = [model2.generate(**tokenizer(t, return_tensors='pt', return_token_type_ids=False),
                max_new_tokens=256,
                early_stopping=True,
                do_sample=True,
                eos_token_id=2,
                ) for t in TEST]
elapsed = (datetime.now()-since).seconds
generated = [tokenizer.decode(x[0]) for x in generated]
for i, j in zip(TEST, generated):
    print("IN: ", end="")
    print(i)
    print("OUT: ", end="")
    print(j)
print(f"Elapsed: {elapsed:04} seconds")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


IN: 대한민국의 수도는 어디인가요?
OUT: 대한민국의 수도는 어디인가요??​​ ​​​​<|endoftext|>
IN: 카페라뗴와 카페오레의 차이점은 무엇인가요?
OUT: 카페라뗴와 카페오레의 차이점은 무엇인가요? -라- 카페오레는 프랑스의 남부 항구 근처에 있는 카페에서 처음 만들어지기 시작한 커피 이름입니다. 라뗴는 커피의 한 분류로 쓰이고 있는 프랑스어입니다. 그래서 커피를 처음 접한 사람들에 게는 카페오레(아메리카노라뗴)를 라떼(Latte)라고 생각하기 쉽습니다. 또 카페오레에는 ‘마들렌’이라는 빵(보통의 빵이 아니라 커피와 곁들여 먹는다네요)이 곁들여지지만 이것이 라뗴가 일반적인 커피가 아님을 보여주기 위해 라떼(Latte)라는 말을 앞에 쓰게 됐습니다. 라- 우유와 설탕을 함께 넣은 커피입니다. 그리고, 에스프레소가 위에 있고 우유를 붓는 것이 라- 우유와 설탕을 넣은 커피라. 맞는 말입니다. 라- 커피에 설탕과 우유 를 넣은 커피인데요, 라- 라떼와 차이는 무엇인가요? -라- 라떼라고 하는 것은 에스프레소 머신을 이용하여 에스프레소를 추출(라떼 아트를 생각해보세요~)하여 우유 를 탄 크림을 듬뿍 넣은 커피를 칭하고요
IN: 영일만친구들은 코딩을 잘 하나요?
OUT: 영일만친구들은 코딩을 잘 하나요?▲ 출처 : https://play.google.com/store/apps/details?id=com.google.android.freeware&hl=ko[출처 : https://goo.gl/4KoFxD][출처 : https://goo.gl/5F3Qcy][출처 : https://goo.gl/MLcnPw]나이키가 코딩에 대해서 고민하는 모습이 보이네요.나이키가 추구하는 운동과 패션에서도 보여지듯이,코딩이 중요한 이유는 소프트웨어를 개발하는 능력을 키워주기 때문인데요.그들은 'STEAM(science,technology,engineering,arts,mathmatics,android)' 을 개발교육 과정에 주입시키고 있는데요.왜 나이키에서는 코딩을 필수로 